In [1]:
%load_ext jupyter_ai_magics

In [2]:
import re
import openai
import os
import sys
import inspect
import pyperclip
import pyautogui
from colored import fg, attr

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def format_python_code(code):
    
    indented_code = ""
    lines = code.split('\n')
    for line in lines:
        indented_code += "    " + line + "\n"  # Add 4 spaces (or a tab) as indentation
    return indented_code


In [4]:
def show_function_definition(function):
   
    source_lines, _ = inspect.getsourcelines(function)
    function_definition = ''.join(source_lines)
    formatted_definition = format_python_code(function_definition)
    
    return formatted_definition


In [5]:
def extract_function_name(text):
    
    match = re.search(r'(\w+)\(', text)
    if match:
        function_name = match.group(1)
        return function_name
    else:
        return ''


def extract_text_between_in_and_newline(text):
    
    start_index = text.find(" in " ) + 4
    end_index = text.find("\n", start_index)
    extracted_text = text[start_index:end_index].strip()
    
    return extract_function_name(extracted_text)


def split_string_last_cell_in(input_string):
    
    index = input_string.rfind("Cell In")
    if index != -1:
        
        stack_trace = input_string[index:].strip()
        
        func_name = extract_text_between_in_and_newline(stack_trace)
        if func_name:
            return func_name
        else:
            return ''
    else:
        return ""

def get_error_function():
    
    function_name = split_string_last_cell_in(Err[list(Err.keys())[-1]])
    if function_name in globals() and callable(globals()[function_name]):
        func = globals()[function_name]
        function_definition = inspect.getsource(func)
        return function_definition
    else:
        print("Function", function_name, "does not exist.")
        return ''


In [6]:
def format_python_code(code):
    keywords = ['and', 'as', 'assert', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except',
                'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'not', 'or', 'pass',
                'raise', 'return', 'try', 'while', 'with', 'yield']

    # Regular expression pattern to match text between quotes
    quotes_pattern = r'([\'"])(.*?)\1'

    formatted_code = ""
    for line in code.split('\n'):
        # Check if the line contains a function definition
        if line.startswith('def'):
            # Extract the function name using regular expression
            match = re.search(r'def\s+(\w+)\s*\(', line)
            if match:
                function_name = match.group(1)
                # Set the color to blue for the function name
                line = line.replace(match.group(1), fg('blue') + function_name + attr('reset'))

        # Process the line and highlight keywords
        for word in line.split():
            if word in keywords:
                # Set the color to green for Python keywords
                line = line.replace(word, fg('green') + word + attr('reset'))

        # Highlight text between quotes in red
        matches = re.findall(quotes_pattern, line)
        for match in matches:
            text_between_quotes = match[1]
            # Set the color to red for the text between quotes
            line = line.replace(match[0] + text_between_quotes + match[0],
                                match[0] + fg('red') + text_between_quotes + attr('reset') + match[0])

        # Highlight function call to `str` in blue
        line = re.sub(r'(\bstr\b)(?=\()', fg('blue') + r'\1' + attr('reset'), line)

        formatted_code += line + '\n'

    return formatted_code


In [7]:
def format_python_code(code):
    keywords = ['and', 'as', 'assert', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except',
                'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'not', 'or', 'pass',
                'raise', 'return', 'try', 'while', 'with', 'yield']

    # Regular expression pattern to match text between quotes
    quotes_pattern = r'([\'"])(.*?)\1'

    formatted_code = ""
    for line in code.split('\n'):
        # Check if the line contains a function definition
        if line.startswith('def'):
            # Extract the function name using regular expression
            match = re.search(r'def\s+(\w+)\s*\(', line)
            if match:
                function_name = match.group(1)
                # Set the color to blue for the function name
                line = line.replace(match.group(1), fg('blue') + function_name + attr('reset'))

        # Process the line and highlight keywords
        for word in line.split():
            if word in keywords:
                # Set the color to green for Python keywords
                line = line.replace(word, fg('green') + word + attr('reset'))

        # Highlight text between quotes in red
        matches = re.findall(quotes_pattern, line)
        for match in matches:
            text_between_quotes = match[1]
            # Set the color to red for the text between quotes
            line = line.replace(match[0] + text_between_quotes + match[0],
                                match[0] + fg('red') + text_between_quotes + attr('reset') + match[0])

        # Highlight function call to `str` in blue
        line = re.sub(r'(\bstr\b)(?=\()', fg('blue') + r'\1' + attr('reset'), line)

        formatted_code += line + '\n'

    return formatted_code

# Example usage
code = '''
def calculate_sum(a, b):
    result = "The sum of a and b is: " + str(a + b)
    message = 'Hello, world!'
    return result, message

def multiply_numbers(x, y):
    product = "The product of x and y is: " + str(x * y)
    return product
'''

formatted_code = format_python_code(code)
print(formatted_code)



def calculate_sum(a, b):
    result = "The sum of a and b is: " + str(a + b)
    message = 'Hello, world!'
    return result, message

def multiply_numbers(x, y):
    product = "The product of x and y is: " + str(x * y)
    return product




In [8]:
def explain_error( model = "text-davinci-003"):
    
    try:
        err_index = list(Err.keys())[-1]
    
    except NameError:
        
        print('Everything is alright, no error to fix')
        return 
    except Exception as e:
        print(e)
        return
    
    else:
        
        code = get_error_function() or In[err_index]
        char_no = 850 + len(Err[err_index]) + len(In[err_index]) # 850 is the length of prompt
        rest_char_no = 4000 - char_no
        prompt = f'Error \"{Err[err_index][-rest_char_no:]} \" was produced from following code \"{code} \" .Explain the error. Rewrite code! Wrap your rewritten code in <<<<< your code >>>>>',

        response = openai.Completion.create(
                  model=model,
                  prompt=prompt,
                  temperature=0,
                  max_tokens=182,
                  top_p=1.0,
                  frequency_penalty=0.0,
                  presence_penalty=0.0,
                  stop=["###"]
                )    
        return response['choices'][0]['text']

In [9]:
def invoke_magic(magic_name, magic_arguments, code):
    """
    Invoke a line or cell magic in Jupyter Notebook.

    Args:
        magic_name (str): Name of the magic, including the leading '%' character.
        magic_arguments (str): Arguments to pass to the magic.
        code (str): Code to execute within the magic.
    """
    if magic_name.startswith("%%"):
       
        return get_ipython().run_cell_magic(magic_name[2:], magic_arguments, code)
    elif magic_name.startswith("%"):
        
        return get_ipython().run_line_magic(magic_name[1:], magic_arguments)
    else:
        raise ValueError("Invalid magic name. Must start with '%%' for cell magic or '%' for line magic.")

def explain_error_with_chatgpt(func=None):
    
    if not func:
        func = In[-2]
    else:
        func = show_function_definition(func)
        
    err_index = list(Err.keys())[-1]
    revised_code = invoke_magic('%%ai','chatgpt',f'Following code \" {func} \" produces error \" {Err[err_index]} \".\
                                 do not Explain the error! Write correct code! Wrap your code in <<< your code >>>' )
    
    return revised_code
            

In [23]:
def revise_code(explanation):
    
    if explanation:
        split_explanation = explanation.split("<<<<<")#" your code >>>>>")
        if len(split_explanation) >= 2:
            revised_code = split_explanation[1].split(">>>>>")[0]
            return revised_code
        else:
            print("Invalid explanation format.")
    else:
        print("No explanation provided.")
    
    return ""  # Return an empty string if no revised code is available


In [11]:
def emulate_typing():
    
    pyautogui.hotkey('esc','b')
    pyautogui.press('enter')
    pyautogui.hotkey('command', 'v')


In [32]:
import logging


Function  does not exist.
 Err[list(Err.keys())[-1]] 



Apply code? Press Enter to apply code or any other key >> Enter to cancel #


In [ ]:

def code_error_fixer():
    # This function tries to fix code errors
    # It does this by first finding the error, then explaining the error, and then suggesting possible solutions
    # If the user chooses to apply the suggested solution, then the solution is copied to the clipboard
    # The user can then paste the solution into their code
    # If the user chooses not to apply the suggested solution, then the solution is not copied to the clipboard

    # Find the error
    # This function returns the error message if it finds an error, or None if it does not find an error
    error_message = find_error()

    # Explain the error
    # This function returns an explanation of the error if it finds an error, or None if it does not find an error
    explanation = explain_error(error_message)

    # Suggest possible solutions
    # This function returns a list of possible solutions if it finds an error, or None if it does not find an error
    suggested_solutions = suggest_solutions(explanation)

    # If there are any suggested solutions, then ask the user if they want to apply them
    if suggested_solutions:
        # Print the suggested solutions
        print(suggested_solutions)

        # Ask the user if they want to apply the suggested solutions
        # This function returns True if the user wants to apply the suggested solutions, or False if they do not
        apply_suggested_solutions = ask_user_if_they_want_to_apply_suggested_solutions()

        # If the user wants to apply the suggested solutions, then copy the suggested solutions to the clipboard
        if apply_suggested_solutions:
            copy_suggested_solutions_to_clipboard(suggested_solutions)

    # If there are no suggested solutions, then print a message to the user
    else:
        print('No suggested solutions')


def code_error_fixer():
    logging.info('Starting code_error_fixer')
    # Get the error and return an explanation
    explained_error = explain_error()
    logging.info('explained_error: {}'.format(explained_error))
    
    # If there is an error, try to fix it
    if explained_error:
        # Return the revised code
        revised_code = revise_code(explained_error)
        logging.info('revised_code: {}'.format(revised_code))
        # If the revised code exists, format it and copy it to the clipboard
        if revised_code:
            print(format_python_code(revised_code))
            pyperclip.copy(revised_code)
            # Ask the user if they want to apply the revised code
            your_choice = input('Apply code? Press Enter to apply code or any other key >> Enter to cancel')
            if not your_choice:
                emulate_typing()
            else:
                pass
        else:
            print('No suggestion')
    else:
        print('No error found')
    logging.info('Ending code_error_fixer')
code_error_fixer()

In [18]:
def c():
    # try to copy the last error message in the Err dict to the clipboard
    try:
        pyperclip.copy(Err[list(Err.keys())[-1]])
    except Exception:
        pass
c()

In [28]:
Err[list(Err.key())[-1]]

AttributeError: 'dict' object has no attribute 'key'